In [2]:
from collections import defaultdict
from graphviz import Digraph, render

star = '*'
line = '|'
dot = '·'
leftBracket, rightBracket = '(', ')'
alphabet = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + \
    [chr(i) for i in range(ord('a'), ord('z') + 1)] + \
    [chr(i) for i in range(ord('0'), ord('9') + 1)]
epsilon = 'ε'

class FA:

    def __init__(self, symbol = set([])):
        self.states = set()
        self.symbol = symbol
        self.transitions = defaultdict(defaultdict)
        self.startstate = None
        self.final_states = []


    def to_str(self):
        num_transitions = (sum(len(transitions) for transitions in self.transitions.values()))
        num_states = len(self.states)
        num_final = len(self.final_states)
        num_symbols = len(self.symbol)
        print(len(self.states))
        return 'states_{}_symbols_{}_transitions_{}_finals_{}'.format(num_states, num_symbols, num_transitions, num_final)

    def setStart(self, state):
        self.startstate = state
        self.states.add(state)

    def addFinal(self, state):
        if isinstance(state, int):
            state = [state]
        for s in state:
            if s not in self.final_states:
                self.final_states.append(s)

    def addTransition(self, fromstate, tostate, inputch):   # add only one
        if isinstance(inputch, str):
            inputch = set([inputch])
        self.states.add(fromstate)
        self.states.add(tostate)
        if fromstate in self.transitions and tostate in self.transitions[fromstate]:
            self.transitions[fromstate][tostate] = \
            self.transitions[fromstate][tostate].union(inputch)
        else:
            self.transitions[fromstate][tostate] = inputch

    def addTransition_dict(self, transitions):  # add dict to dict
        for fromstate, tostates in transitions.items():
            for state in tostates:
                self.addTransition(fromstate, state, tostates[state])

    def newBuildFromNumber(self, startnum):
        translations = {}
        for i in self.states:
            translations[i] = startnum
            startnum += 1
        rebuild = FA(self.symbol)
        rebuild.setStart(translations[self.startstate])
        rebuild.addFinal(translations[self.final_states[0]])
        for fromstate, tostates in self.transitions.items():
            for state in tostates:
                rebuild.addTransition(translations[fromstate], translations[state], tostates[state])
        return [rebuild, startnum]

    def newBuildFromEqualStates(self, equivalent, pos):
        # change states' number after merging
        rebuild = FA(self.symbol)
        for fromstate, tostates in self.transitions.items():
            for state in tostates:
                rebuild.addTransition(pos[fromstate], pos[state], tostates[state])
        rebuild.setStart(pos[self.startstate])
        for s in self.final_states:
            rebuild.addFinal(pos[s])
        return rebuild

    def getEpsilonClosure(self, findstate):
        allstates = set()
        states = [findstate]
        while len(states):
            state = states.pop()
            allstates.add(state)
            if state in self.transitions:
                for tos in self.transitions[state]:
                    if epsilon in self.transitions[state][tos] and tos not in allstates:
                        states.append(tos)
        return allstates

    def getMove(self, state, skey):
        if isinstance(state, int):
            state = [state]
        trstates = set()
        for st in state:
            if st in self.transitions:
                for tns in self.transitions[st]:
                    if skey in self.transitions[st][tns]:
                        trstates.add(tns)
        return trstates

    def display(self, fname, pname):
        fa = Digraph(pname, filename = fname, format = 'png')
        fa.attr(rankdir='LR')

        fa.attr('node', shape = 'doublecircle')
        for fst in self.final_states:
            fa.node('s' + str(fst))

        fa.attr('node', shape = 'circle')
        for fromstate, tostates in self.transitions.items():
            for state in tostates:
                tmp = ''
                for s in tostates[state]:
                    tmp += s + '|'
                fa.edge('s' + str(fromstate), 's' + str(state), label = tmp[:-1])

        fa.attr('node', shape = 'point')
        fa.edge('', 's' + str(self.startstate))

        fa.view()

class Regex2NFA:

    def __init__(self, regex):
        self.regex = regex
        self.buildNFA()

    def displayNFA(self):
        self.nfa.display('nfa_{}.gv'.format(self.regex), 'nondeterministic_finite_state_machine')

    @staticmethod
    def getPriority(op):
        if op == line:
            return 1
        elif op == dot:
            return 2
        elif op == star:
            return 3
        else:       # left bracket
            return 0

    @staticmethod
    def basicstruct(inputch):   # Regex = a -> NFA
        state1 = 1
        state2 = 2
        basic = FA(set([inputch]))
        basic.setStart(state1)
        basic.addFinal(state2)
        basic.addTransition(state1, state2, inputch)
        return basic

    @staticmethod
    def linestruct(a, b):   # Regex = a | b -> NFA
        [a, m1] = a.newBuildFromNumber(2)
        [b, m2] = b.newBuildFromNumber(m1)
        state1 = 1
        state2 = m2
        lineFA = FA(a.symbol.union(b.symbol))
        lineFA.setStart(state1)
        lineFA.addFinal(state2)
        lineFA.addTransition(lineFA.startstate, a.startstate, epsilon)
        lineFA.addTransition(lineFA.startstate, b.startstate, epsilon)
        lineFA.addTransition(a.final_states[0], lineFA.final_states[0], epsilon)
        lineFA.addTransition(b.final_states[0], lineFA.final_states[0], epsilon)
        lineFA.addTransition_dict(a.transitions)
        lineFA.addTransition_dict(b.transitions)
        return lineFA

    @staticmethod
    def dotstruct(a, b):    # Regex = a · b -> NFA
        [a, m1] = a.newBuildFromNumber(1)
        [b, m2] = b.newBuildFromNumber(m1)
        state1 = 1
        state2 = m2 - 1
        dotFA = FA(a.symbol.union(b.symbol))
        dotFA.setStart(state1)
        dotFA.addFinal(state2)
        dotFA.addTransition(a.final_states[0], b.startstate, epsilon)
        dotFA.addTransition_dict(a.transitions)
        dotFA.addTransition_dict(b.transitions)
        return dotFA

    @staticmethod
    def starstruct(a):  # Regex = a* -> NFA
        [a, m1] = a.newBuildFromNumber(2)
        state1 = 1
        state2 = m1
        starFA = FA(a.symbol)
        starFA.setStart(state1)
        starFA.addFinal(state2)
        starFA.addTransition(starFA.startstate, a.startstate, epsilon)
        starFA.addTransition(starFA.startstate, starFA.final_states[0], epsilon)
        starFA.addTransition(a.final_states[0], starFA.final_states[0], epsilon)
        starFA.addTransition(a.final_states[0], a.startstate, epsilon)
        starFA.addTransition_dict(a.transitions)
        return starFA

    def buildNFA(self):
        tword = ''
        pre = ''
        symbol = set()

        for ch in self.regex:
            if ch in alphabet:
                symbol.add(ch)
            if ch in alphabet or ch == leftBracket:
                if pre != dot and (pre in alphabet or pre in [star, rightBracket]):
                    tword += dot
            tword += ch
            pre = ch
        self.regex = tword

        tword = ''
        stack = []
        for ch in self.regex:
            if ch in alphabet:
                tword += ch
            elif ch == leftBracket:
                stack.append(ch)
            elif ch == rightBracket:
                while(stack[-1] != leftBracket):
                    tword += stack[-1]
                    stack.pop()
                stack.pop()
            else:
                while(len(stack) and Regex2NFA.getPriority(stack[-1]) >= Regex2NFA.getPriority(ch)):
                    tword += stack[-1]
                    stack.pop()
                stack.append(ch)
        while(len(stack) > 0):
            tword += stack.pop()
        self.regex = tword

        # build ε-NFA from postfix expression
        self.automata = []
        for ch in self.regex:
            if ch in alphabet:
                self.automata.append(Regex2NFA.basicstruct(ch))
            elif ch == line:
                b = self.automata.pop()
                a = self.automata.pop()
                self.automata.append(Regex2NFA.linestruct(a, b))
            elif ch == dot:
                b = self.automata.pop()
                a = self.automata.pop()
                self.automata.append(Regex2NFA.dotstruct(a, b))
            elif ch == star:
                a = self.automata.pop()
                self.automata.append(Regex2NFA.starstruct(a))
        self.nfa = self.automata.pop()
        self.nfa.symbol = symbol

class NFA2DFA:

    def __init__(self, nfa):
        self.buildDFA(nfa)

    def displayDFA(self):
        self.dfa.display('dfa.gv', 'deterministic_finite_state_machine')

    def displayminDFA(self, directory, file_name):
        self.minDFA.display('{}/mindfa_{}.gv'.format(directory, file_name), 'min_deterministic_finite_state_machine')

    def buildDFA(self, nfa):    # subset method
        allstates = dict()  # visited subset
        eclosure = dict()   # every state's ε-closure
        state1 = nfa.getEpsilonClosure(nfa.startstate)
        eclosure[nfa.startstate] = state1
        cnt = 1 # the number of subset, dfa state id
        dfa = FA(nfa.symbol)
        dfa.setStart(cnt)
        states = [[state1, dfa.startstate]] # unvisit
        allstates[cnt] = state1
        cnt += 1
        while len(states):
            [state, fromindex] = states.pop()
            for ch in dfa.symbol:
                trstates = nfa.getMove(state, ch)
                for s in list(trstates):
                    if s not in eclosure:
                        eclosure[s] = nfa.getEpsilonClosure(s)
                    trstates = trstates.union(eclosure[s])
                if len(trstates):
                    if trstates not in allstates.values():
                        states.append([trstates, cnt])
                        allstates[cnt] = trstates
                        toindex = cnt
                        cnt += 1
                    else:
                        toindex = [k for k, v in allstates.items() if v  ==  trstates][0]
                    dfa.addTransition(fromindex, toindex, ch)
            for value, state in allstates.items():
                if nfa.final_states[0] in state:
                    dfa.addFinal(value)
            self.dfa = dfa

    @staticmethod
    def reNumber(states, pos):  # renumber the sets' number begin from 1
        cnt = 1
        change = dict()
        for st in states:
            if pos[st] not in change:
                change[pos[st]] = cnt
                cnt += 1
            pos[st] = change[pos[st]]

    def minimise(self):
        states = list(self.dfa.states)
        tostate = dict(set())

        for st in states:
            for sy in self.dfa.symbol:
                if st in tostate:
                    if sy in tostate[st]:
                        tostate[st][sy] = tostate[st][sy].union(self.dfa.getMove(st, sy))
                    else:
                        tostate[st][sy] = self.dfa.getMove(st, sy)
                else:
                    tostate[st] = {sy : self.dfa.getMove(st, sy)}
                if len(tostate[st][sy]):
                    tostate[st][sy] = tostate[st][sy].pop()
                else:
                    tostate[st][sy] = 0

        equal = dict()
        pos = dict()

        # initialization 2 sets, nonterminal states and final states
        equal = {1: set(), 2: set()}
        for st in states:
            if st not in self.dfa.final_states:
                equal[1] = equal[1].union(set([st]))
                pos[st] = 1
        for fst in self.dfa.final_states:
            equal[2] = equal[2].union(set([fst]))
            pos[fst] = 2

        unchecked = []
        cnt = 3 # the number of sets
        unchecked.extend([[equal[1], 1], [equal[2], 2]])
        while len(unchecked):
            [equalst, id] = unchecked.pop()
            for sy in self.dfa.symbol:
                diff = dict()
                for st in equalst:
                    if tostate[st][sy] == 0:
                        if 0 in diff:
                            diff[0].add(st)
                        else:
                            diff[0] = set([st])
                    else:
                        if pos[tostate[st][sy]] in diff:
                            diff[pos[tostate[st][sy]]].add(st)
                        else:
                            diff[pos[tostate[st][sy]]] = set([st])
                if len(diff) > 1:
                    for k, v in diff.items():
                        if k:
                            for i in v:
                                equal[id].remove(i)
                                if cnt in equal:
                                    equal[cnt] = equal[cnt].union(set([i]))
                                else:
                                    equal[cnt] = set([i])
                            if len(equal[id]) == 0:
                                equal.pop(id)
                            for i in v:
                                pos[i] = cnt
                            unchecked.append([equal[cnt], cnt])
                            cnt += 1
                    break
        if len(equal) == len(states):
            self.minDFA = self.dfa
        else:
            NFA2DFA.reNumber(states, pos)
            self.minDFA = self.dfa.newBuildFromEqualStates(equal, pos)

    def analysis(self, string):
        string = string.replace('@', epsilon)
        curst = self.dfa.startstate
        for ch in string:
            if ch == epsilon:
                continue
            st = list(self.dfa.getMove(curst, ch))
            if len(st) == 0:
                return False
            curst = st[0]
        if curst in self.dfa.final_states:
            return True
        return False


def regex_to_fsm(regex: str):
    a = Regex2NFA(regex)
    #a.displayNFA()

    b = NFA2DFA(a.nfa)
    #b.displayDFA()
    b.minimise()
    mdfa_str = b.minDFA.to_str()
    folder_path = "/datadrive/formal_language/{}".format(mdfa_str)

    b.displayminDFA(folder_path, regex)
    return b.minDFA


In [128]:
#regex = '(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z)*'
regex = '((0|1|2|3|4|5|6|7|8|9|10)*|XYZ)(cd|bc|de|fg|gh|hi|jk|kl|lm|mn|no|pq|qr|rs|st|tu|v|w|x|y|z)(a*|e*|i*|o*|u*)(A|B|C|D|E|F|G)*'
a = Regex2NFA(regex)
#a.displayNFA()
b = NFA2DFA(a.nfa)
print(b.analysis('baaaat'))

b.minimise()
print(b.minDFA.states)
print(b.minDFA.symbol)

print(b.minDFA.transitions)
b.minDFA.to_str()

False
{1, 2, 3, 4, 5, 6}
{'9', '8', 'G', 'r', 'l', 'j', '3', 'D', 'c', 'F', '4', 'X', 'd', 'q', 'g', '5', 'o', '2', 'e', 'a', 'v', 'C', 'b', 'm', '7', 'p', 'w', 'y', 'B', 't', 'Z', 's', '1', 'h', 'f', 'z', 'i', 'u', 'n', '6', 'Y', 'k', 'A', 'x', '0', 'E'}
defaultdict(<class 'collections.defaultdict'>, {1: defaultdict(None, {2: {'9', '8', '5', '3', '2', '6', '0', '1', '7', '4'}, 3: {'r', 'l', 'j', 't', 'c', 's', 'h', 'X', 'f', 'd', 'q', 'g', 'n', 'k', 'b', 'm', 'p'}, 4: {'w', 'z', 'x', 'v', 'y'}}), 2: defaultdict(None, {2: {'9', '8', '5', '3', '2', '6', '0', '1', '7', '4'}, 3: {'r', 'l', 'j', 't', 'c', 's', 'h', 'f', 'd', 'q', 'g', 'n', 'k', 'b', 'm', 'p'}, 4: {'w', 'z', 'x', 'v', 'y'}}), 4: defaultdict(None, {5: {'G', 'i', 'B', 'u', 'e', 'D', 'a', 'A', 'E', 'C', 'F'}, 6: {'o'}}), 5: defaultdict(None, {5: {'G', 'i', 'B', 'u', 'e', 'D', 'A', 'E', 'a', 'C', 'F'}}), 6: defaultdict(None, {5: {'D', 'A', 'G', 'E', 'B', 'C', 'F'}, 6: {'o'}}), 3: defaultdict(None, {4: {'w', 'r', 'l', 'y', 't', 

'states_6_symbols_46_transitions_13_finals_3'

In [117]:
import exrex
limit = 100
word_count = exrex.count(regex, limit=limit)
word_count

10500

In [118]:
import random
generated = list(exrex.generate(regex, limit=2))
random.shuffle(generated)
generated

['1hi00',
 '1pq00',
 '1tuo00',
 '1rsi00',
 '1lmo00',
 '1tu00',
 '1qru00',
 '1cd00',
 '1de00',
 '1z00',
 '1deo00',
 '1hi00',
 '1mno00',
 '1qra00',
 '1qr00',
 '1cda00',
 '1hio00',
 '1ste00',
 '1lma00',
 '1kl00',
 '1fg00',
 '1y00',
 '1st00',
 '1x00',
 '1jk00',
 '1xa00',
 '1mn00',
 '1gh00',
 '1tuu00',
 '1rsa00',
 '1noe00',
 '1kli00',
 '1no00',
 '1lmi00',
 '1pq00',
 '1ghu00',
 '1w00',
 '1x00',
 '1bc00',
 '1qri00',
 '1tu00',
 '1wo00',
 '1stu00',
 '1jka00',
 '1cd00',
 '1pqu00',
 '1sto00',
 '1lm00',
 '1bca00',
 '1gh00',
 '1lm00',
 '1kle00',
 '1ye00',
 '1xi00',
 '1dei00',
 '1hia00',
 '1wa00',
 '1qr00',
 '1zi00',
 '1ghi00',
 '1cd00',
 '1wu00',
 '1lme00',
 '1jk00',
 '1ghe00',
 '1kl00',
 '1yu00',
 '1ve00',
 '1w00',
 '1klo00',
 '1zu00',
 '1cdi00',
 '1rsu00',
 '1de00',
 '1za00',
 '1bcu00',
 '1mn00',
 '1z00',
 '1fg00',
 '1mn00',
 '1cdu00',
 '1fgi00',
 '1bco00',
 '1tua00',
 '1st00',
 '1y00',
 '1no00',
 '1lmu00',
 '1jki00',
 '1kl00',
 '1rso00',
 '1v00',
 '1xu00',
 '1fgu00',
 '1rs00',
 '1deu00',
 '1st00

In [119]:
print(regex)
mdfa = regex_to_fsm(regex)
print(mdfa.states)
folder_path= mdfa.to_str()
folder_path

1(cd|bc|de|fg|gh|hi|jk|kl|lm|mn|no|pq|qr|rs|st|tu|v|w|x|y|z)(a*|e*|i*|o*|u*)00
4
{1, 2, 3, 4}
4


'states_4_symbols_28_transitions_6_finals_1'

In [120]:
import os
os.mkdir(folder_path)


In [121]:
from IPython.display import Image
graph_file_path = "/datadrive/formal_language/{}/mindfa_{}.gv.png".format(folder_path, regex)

Image(filename=graph_file_path) 

In [122]:
info_file_path = "/datadrive/formal_language/{}/info.txt".format(folder_path)
with open(info_file_path, 'w+') as info_file:
    info_file.write('regex={} \nstates: {} \naphabet: {} \ntransitions: {} \nfinals: {}'.format(regex, mdfa.states, mdfa.symbol, mdfa.transitions, mdfa.final_states ))


In [123]:
files = []
file_names = []
for i in range(4):
    file_path = "/datadrive/formal_language/{}/{}_shard_shuff.txt".format(folder_path, i)
    files.append(open(file_path, 'w+'))
    file_names.append(file_path)
for i in range(1):
    file_path = "/datadrive/formal_language/{}/{}_val_shard_shuff.txt".format(folder_path, i)
    files.append(open(file_path, 'w+'))
    file_names.append(file_path)

    
files

[<_io.TextIOWrapper name='/datadrive/formal_language/states_4_symbols_28_transitions_6_finals_1/0_shard_shuff.txt' mode='w+' encoding='UTF-8'>,
 <_io.TextIOWrapper name='/datadrive/formal_language/states_4_symbols_28_transitions_6_finals_1/1_shard_shuff.txt' mode='w+' encoding='UTF-8'>,
 <_io.TextIOWrapper name='/datadrive/formal_language/states_4_symbols_28_transitions_6_finals_1/2_shard_shuff.txt' mode='w+' encoding='UTF-8'>,
 <_io.TextIOWrapper name='/datadrive/formal_language/states_4_symbols_28_transitions_6_finals_1/3_shard_shuff.txt' mode='w+' encoding='UTF-8'>,
 <_io.TextIOWrapper name='/datadrive/formal_language/states_4_symbols_28_transitions_6_finals_1/0_val_shard_shuff.txt' mode='w+' encoding='UTF-8'>]

In [124]:
gen = exrex.generate(regex, limit=limit)
word_count
for i in range(word_count):
    word = next(gen)
    file_index = i % len(files)
    f = files[file_index]
    f.write("%s\n" % word)
for file in files:
    file.close()
        

In [125]:
import random
for file_name in file_names:
    lines = open(file_name).readlines()
    random.shuffle(lines)
    open(file_name, 'w').writelines(lines)

In [126]:
print(folder_path)
print(regex)

states_4_symbols_28_transitions_6_finals_1
1(cd|bc|de|fg|gh|hi|jk|kl|lm|mn|no|pq|qr|rs|st|tu|v|w|x|y|z)(a*|e*|i*|o*|u*)00
